In [1]:
import os
import numpy as np
import pandas as pd
import pickle
import plotly.express as px

In [8]:
df = pd.read_pickle("large_data/climate_mentions.pkl")
df.set_index(['Year', 'ISO-alpha3 Code'], inplace=True)

df = df.sort_values(['Climate Mentions Count'], ascending=False)
df['Climate Mentions Count - Absolute'] = df['Climate Mentions Count'] * df['Speech'].str.len()
df.head()

,,Session,Speech,Climate Mentions Count,Climate Mentions Count - Absolute
Year,ISO-alpha3 Code,,,,
2016,LTU,71,This year we agreed that the general debate wo...,0.529588,2300.0
2015,WSM,70,Last Friday ushered in a new era in our search...,0.342249,5400.0
1977,WSM,32,"﻿1.\tFirst I must extend to you, Sir, the warm...",0.331078,4600.0
1975,LAO,30,1. It is for me a great privilege to represent...,0.321007,5000.0
2019,FIN,74,Let me congratulate Mr. Tijjani Muhammad-Bande...,0.312969,2500.0


In [9]:
# can't take the average of tempereture anomaly, difference in area etc, so a different dataset for total global warming
world = pd.read_csv('./data/globaltemp.csv')

#create extra column that gives only the year of the anomaly per month
world['Year'] = pd.DatetimeIndex(world['Day']).year

#change column names & remove 'Code'
world = world.rename(columns = {'Entity': 'Country', 'temperature_anomaly': 'Anomaly'})[['Country', 'Anomaly', 'Year']]

# date is per month, change it to anomaly per year, set index to year
world = world[world['Country'] == 'World']
world = world.groupby('Year').mean()

#only get the data from 1970 and up, because of our main data set being 1970 and higher
world = world[world.index > 1969]

In [10]:
world.head()

,Anomaly
Year,
1970,0.025000
1971,-0.084167
1972,0.008333
1973,0.160000
1974,-0.070000


In [17]:
world_df = df.groupby('Year').mean()
world_df['Anomaly'] = world['Anomaly']
world_df.head()
world_df.reset_index()

,Year,Session,Climate Mentions Count,Climate Mentions Count - Absolute,Anomaly
0,1970,25.0,0.025946,638.571429,0.025000
1,1971,26.0,0.026749,633.620690,-0.084167
2,1972,27.0,0.029312,716.000000,0.008333
3,1973,28.0,0.029853,768.333333,0.160000
4,1974,29.0,0.030554,745.736434,-0.070000
5,1975,30.0,0.044799,1053.968254,-0.013333
6,1976,31.0,0.038799,980.597015,-0.096667
7,1977,32.0,0.040237,960.714286,0.176667
8,1978,33.0,0.037759,920.567376,0.068333
9,1979,34.0,0.036890,960.416667,0.165000


In [24]:
fig = px.scatter(world_df, x=np.arange(1970, 2021), y='Climate Mentions Count', color="Anomaly", color_continuous_scale='Inferno')
fig.update_layout(
    title=f"Climate mentions in UN Speeches and global temperature anomaly",
    xaxis_title="Year",
)

fig.show()

In [25]:
fig.write_html("exports/html/climate_mentions_and_temperature_anomaly.html")

In [26]:
import plotly.io as pio
pio.kaleido.scope.default_format = "svg"
pio.kaleido.scope.default_scale = 2

fig.write_image("exports/images/climate_mentions_and_temperature_anomaly.svg")